In [ ]:
import sqlite3
import csv
import os

### Set Database Update Values

In [47]:
def update_database():
    conn = sqlite3.connect('cyclist_database.db')
    cursor = conn.cursor()

    cursor.execute("UPDATE User SET first_name = ?, last_name = ?, email = ?, password = ? WHERE id = ?", ('David', 'Naruto', 'davidnaruto@simplon.co', 'cyclist_legend', 1))
    cursor.execute("UPDATE User SET first_name = ?, last_name = ?, email = ?, password = ? WHERE id = ?", ('Malek', 'Nara', 'maleknara@simplon.co', 'cyclist_nara', 2))
    cursor.execute("UPDATE User SET first_name = ?, last_name = ?, email = ?, password = ? WHERE id = ?", ('Gauthier', 'Obito', 'gauthierobito@simplon.co', 'cyclist_yamanaka', 3))
    cursor.execute("UPDATE User SET first_name = ?, last_name = ?, email = ?, password = ? WHERE id = ?", ('Ludivine', 'Sakura', 'ludivinesakura@simplon.co', 'cyclist_uzumaki', 4))
    cursor.execute("UPDATE User SET first_name = ?, last_name = ?, email = ?, password = ? WHERE id = ?", ('Nicolas', 'Madara', 'nicolasmadara@simplon.co', 'cyclist_uchiha', 5))
    cursor.execute("UPDATE User SET first_name = ?, last_name = ?, email = ?, password = ? WHERE id = ?", ('Leo', 'Gara', 'leogara@simplon.co', 'cyclist_hozuki', 6))
    cursor.execute("UPDATE User SET first_name = ?, last_name = ?, email = ?, password = ? WHERE id = ?", ('Maxime', 'Tobirama', 'maximetobirama@simplon.co', 'cyclist_senju', 7))
        
        
    cyclist_info_updates = [
            ('male', 22.1, 67.9, 185, 1),
            ('male', 25.6, 79.2, 182.4, 2),
            ('male', 42.8, 81.9, 173.8, 3),
            ('female', 21.6, 66.2, 165.9, 4),
            ('male', 46.6, 81.1, 181, 5),
            ('male', 28.8, 74.4, 171, 6),
            ('male', 41.2, 81.1, 180.5, 7)
        ]
    for gender, age, weight, height, user_id in cyclist_info_updates:
        cursor.execute("UPDATE CyclistInfo SET gender = ?, age = ?, weight = ?, height = ? WHERE user_id = ?", (gender, age, weight, height, user_id))

    conn.commit()
    conn.close()

### Create Database

In [ ]:
import sqlite3
import csv
import os

def main():
    conn = sqlite3.connect('cyclist_database.db')
    cursor = conn.cursor()

    cursor.executescript('''
        DROP TABLE IF EXISTS User;
        CREATE TABLE User(
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            email TEXT NOT NULL,
            password TEXT NOT NULL,
            is_staff BOOLEAN NOT NULL DEFAULT 0
        );
        
        DROP TABLE IF EXISTS CyclistInfo;
        CREATE TABLE CyclistInfo(
            user_id INTEGER PRIMARY KEY,
            gender TEXT CHECK(gender IN ('male', 'female')) NOT NULL,
            age INTEGER CHECK(age > 0) NOT NULL,
            weight REAL NOT NULL,
            height REAL NOT NULL,
            FOREIGN KEY(user_id) REFERENCES User(id)
        );
                         
        DROP TABLE IF EXISTS Test;
        CREATE TABLE Test(
            test_id INTEGER PRIMARY KEY AUTOINCREMENT,
            test_type TEXT NOT NULL UNIQUE
        );
                                   
        DROP TABLE IF EXISTS Performance;
        CREATE TABLE Performance(
            performance_id INTEGER PRIMARY KEY AUTOINCREMENT,
            test_id INTEGER NOT NULL,
            user_id INTEGER NOT NULL, 
            time INTEGER NOT NULL,
            power REAL NOT NULL,
            oxygen INTEGER NOT NULL,
            cadence REAL NOT NULL,
            heart_rate REAL NOT NULL,
            respiration_frequency REAL NOT NULL,
            FOREIGN KEY(test_id) REFERENCES Test(test_id),
            FOREIGN KEY(user_id) REFERENCES User(id)
        );
    ''')

    # Prepopulate Test table with the four unique test types.
    test_types = ['i', 'ii', 'incremental', 'wingate']
    for t in test_types:
        cursor.execute("INSERT OR IGNORE INTO Test (test_type) VALUES (?)", (t,))

    # Process CSV files found in the "data" directory.
    for file in os.listdir('data'):
        if file.endswith('.csv'):
            file_path = os.path.join('data', file)
            print('Importing', file)

            # Expect filename format: sbj_<user_id>_<test_type>.csv
            filename_without_ext = os.path.splitext(file)[0]
            parts = filename_without_ext.split('_')
            if len(parts) < 3:
                print("Filename format not recognized:", file)
                continue

            try:
                user_id = int(parts[1])
            except ValueError:
                print("User id not an integer in file:", file)
                continue

            test_type = parts[2].lower()
            if test_type not in test_types:
                print("Test type not recognized:", test_type)
                continue


            cursor.execute("INSERT OR IGNORE INTO User (id, first_name, last_name, email, password, is_staff) VALUES (?, '', '', '', '', 0)", (user_id,))
            cursor.execute("INSERT OR IGNORE INTO CyclistInfo (user_id, gender, age, weight, height) VALUES (?, 'male', 18, 0.0, 0.0)", (user_id,))

            # Retrieve the test_id for the given test_type.
            cursor.execute("SELECT test_id FROM Test WHERE test_type = ?", (test_type,))
            row = cursor.fetchone()
            if row is None:
                print("Error retrieving test_id for", test_type)
                continue
            test_id = row[0]

            with open(file_path, 'r', encoding='utf-8') as f:
                reader = csv.reader(f)
                next(reader, None)  # Skip header row if it exists.
                for row in reader:
                    cursor.execute(
                        "INSERT INTO Performance(test_id, user_id, time, oxygen, power, cadence, heart_rate, respiration_frequency) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                        (test_id, user_id, row[0], row[1], row[2], row[3], row[4], row[5])
                    )

    conn.commit()
    conn.close()

if __name__ == '__main__':
    main()
    update_database()

Importing sbj_6_Wingate.csv
Importing sbj_6_I.csv
Importing sbj_7_incremental.csv
Importing sbj_4_I.csv
Importing sbj_2_II.csv
Importing sbj_2_I.csv
Importing sbj_3_Wingate.csv
Importing sbj_5_I.csv
Importing sbj_6_II.csv
Importing sbj_1_incremental.csv
Importing sbj_7_I.csv
Importing sbj_3_I.csv
Importing sbj_6_incremental.csv
Importing sbj_1_I.csv
Importing sbj_4_II.csv
Importing sbj_5_Wingate.csv
Importing sbj_3_incremental.csv
Importing sbj_1_II.csv
Importing sbj_2_Wingate.csv
Importing sbj_7_Wingate.csv
Importing sbj_4_incremental.csv
Importing sbj_3_II.csv
Importing sbj_4_Wingate.csv
Importing sbj_7_II.csv
Importing sbj_5_incremental.csv
Importing sbj_5_II.csv
Importing sbj_2_incremental.csv
Importing sbj_1_Wingate.csv


In [ ]:
import pandas as pd

conn = sqlite3.connect('cyclist_database.db')
cursor =  conn.cursor()

df = pd.read_sql_query("SELECT User.id, User.first_name, User.last_name, User.email, User.password, User.is_staff,"
                               "CyclistInfo.gender, CyclistInfo.age, CyclistInfo.weight, CyclistInfo.height,"
                                "Performance.time, Performance.power, Performance.oxygen, Performance.cadence, Performance.heart_rate, Performance.respiration_frequency,"
                                "Test.test_type "
                        "FROM USER "
                        "LEFT JOIN CyclistInfo ON User.id = CyclistInfo.user_id "
                        "LEFT JOIN PERFORMANCE ON CyclistInfo.user_id = performance.user_id "
                        "LEFT JOIN TEST ON performance.test_id = test.test_id", conn)

df.head(20)

,id,first_name,last_name,email,password,is_staff,gender,age,weight,height,time,power,oxygen,cadence,heart_rate,respiration_frequency,test_type
0,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,1,318.400000,0.0,0.0,75.600000,20.100000,i
1,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,2,356.166667,0.0,0.0,75.666667,19.750000,i
2,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,3,403.285714,0.0,0.0,75.714286,19.428571,i
3,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,4,456.250000,0.0,0.0,75.750000,19.125000,i
4,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,5,478.925926,0.0,0.0,75.740741,18.962963,i
5,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,6,480.400000,0.0,0.0,75.700000,18.900000,i
6,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,7,480.000000,0.0,0.0,75.700000,18.600000,i
7,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,8,482.716667,0.0,0.0,75.691667,18.375000,i
8,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,9,488.550000,0.0,0.0,75.675000,18.225000,i
9,1,David,Naruto,davidnaruto@simplon.co,cyclist_legend,0,male,22.1,67.9,185.0,10,497.500000,0.0,0.0,75.650000,18.150000,i
